<h1><center> Foursquare Location Matching </center></h1>
<h3><center> Model Selection </center></h3>
<h3><center> Tao Shan </center></h3>

This notebook compares different models and choose the best one for future use.

### Other Relevant notebooks and links

Competition: [Foursquare - Location Matching](https://www.kaggle.com/competitions/foursquare-location-matching)

Train data generation notebook: [Foursquare - train data generation](https://www.kaggle.com/taos2000/foursquare-train-data-generation)

Train data preprocessing notebook: [Foursquare - train data preprocess](https://www.kaggle.com/taos2000/foursquare-train-data-preprocess)

Model Selection: [Foursquare - model selection](https://www.kaggle.com/taos2000/foursquare-model-selection)

Model Training: [Foursquare - model training](https://www.kaggle.com/taos2000/foursquare-model-training)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from pickle import dump
import gc
import time
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

from geopy.geocoders import Nominatim

import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer

import Levenshtein as lev
import math
from collections import Counter

from pickle import dump, load
import time
from sklearn.neighbors import BallTree


import itertools
from tqdm.auto import tqdm
tqdm.pandas()
import gc

from fuzzywuzzy import fuzz
from xgboost import XGBClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel

In [2]:
train_pairs = pd.read_pickle('../input/last-day-four-points/train_pairs (1).pkl').sample(100000,random_state = 8888)
test_pairs = pd.read_pickle('../input/last-day-four-points/train_pairs_test.pkl').sample(100000,random_state = 8888)
sample_pairs = pd.read_pickle('../input/last-day-four-points/train_pairs_sample.pkl')

In [3]:
cols = ['address_1_missing',
        'country_same',
 'address_2_missing',
 'categories_1_count',
 'categories_1_count_diff',
 'categories_2_count',
 'city_1_count',
 'city_1_count_diff',
 'city_1_missing',
 'city_2_count',
 'city_2_missing',
 'country_1_count',
 'country_1_count_diff',
 'distance',
 'latitude_1',
 'latitude_1_count',
 'latitude_2_count',
 'latitude_2_count_diff',
 'longitude_1',
 'longitude_1_count',
 'longitude_2_count',
 'longitude_2_count_diff',
 'phone_1_missing',
 'phone_2_missing',
 'state_1_count',
 'state_1_count_diff',
 'state_2_count',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']
ids = ['id_1','id_2']
cols_new = ['address_1_missing',
 'address_2_missing',
 'city_1_missing',
 'city_2_missing',
 'distance',
 'latitude_1',
 'longitude_1',
 'phone_1_missing',
 'phone_2_missing',
 'url_1_missing',
 'url_2_missing',
 'zip_1_missing',
 'zip_2_missing',
 'name_1_fuzzy',
 'name_1_fuzzy_partial',
 'address_1_fuzzy',
 'categories_1_fuzzy',
 'categories_1_fuzzy_partial',
 'url_1_fuzzy']

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
model = GradientBoostingClassifier()
selector = SelectFromModel(model).fit(X_train, y_train)

In [5]:
X_train.columns[selector.get_support()]

Index(['country_same', 'distance', 'name_1_fuzzy', 'name_1_fuzzy_partial',
       'categories_1_fuzzy_partial'],
      dtype='object')

XGboost: ['country_same', 'address_2_missing', 'distance', 'name_1_fuzzy',
       'name_1_fuzzy_partial', 'categories_1_fuzzy_partial'],

random forest: ['country_same', 'distance', 'name_1_fuzzy', 'name_1_fuzzy_partial',
       'categories_1_fuzzy', 'categories_1_fuzzy_partial']
       
gradient boost: ['country_same', 'distance', 'name_1_fuzzy', 'name_1_fuzzy_partial',
       'categories_1_fuzzy_partial']

In [6]:
select_col = ['country_same', 'address_2_missing', 'distance', 'name_1_fuzzy', 'name_1_fuzzy_partial', 'categories_1_fuzzy_partial','categories_1_fuzzy']

In [7]:
X_train = train_pairs[select_col]; y_train = train_pairs['match']
X_test = test_pairs[select_col]; y_test = test_pairs['match']

xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train,y_train)
test_pred = xgb.predict_proba(X_test)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7048283094555172


In [8]:
X_train

,country_same,address_2_missing,distance,name_1_fuzzy,name_1_fuzzy_partial,categories_1_fuzzy_partial,categories_1_fuzzy
810264,1,0,156.466048,1.000000,100,0,0.000000
719720,1,0,6450.172502,0.606061,61,100,1.000000
4783878,1,0,754.655290,1.000000,100,100,1.000000
4459094,1,0,3.538756,0.454545,56,0,0.000000
3258944,1,1,7.066355,0.432432,41,29,0.400000
...,...,...,...,...,...,...,...
204332,0,0,5739.824595,0.000000,0,56,0.300000
2415322,1,1,1.495704,0.000000,0,73,0.615385
230261,1,1,9.754244,0.176471,29,20,0.166667
1205744,0,1,11649.888515,0.090909,25,41,0.303030


In [9]:
X_test

,country_same,address_2_missing,distance,name_1_fuzzy,name_1_fuzzy_partial,categories_1_fuzzy_partial,categories_1_fuzzy
444919,1,1,4.574214,0.437500,60,43,0.285714
221617,1,0,20.622459,0.437500,67,0,0.000000
108116,1,0,4464.345963,0.375000,40,0,0.000000
455951,1,1,2.615121,0.384615,45,33,0.307692
544395,1,0,21.179323,0.950000,95,100,0.625000
...,...,...,...,...,...,...,...
229822,1,1,7.348442,0.411765,50,100,0.745098
253314,1,1,219.908745,1.000000,100,100,0.472222
568951,1,0,5.401779,0.692308,75,22,0.222222
503576,1,1,57.535024,1.000000,100,100,0.745098


In [10]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']

xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train,y_train)
test_pred = xgb.predict_proba(X_test)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7718455527647068


In [11]:
scaler = MinMaxScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7718406418420891


In [12]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7735606372015167


In [13]:
X_train = train_pairs[cols_new]; y_train = train_pairs['match']
X_test = test_pairs[cols_new]; y_test = test_pairs['match']

xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train,y_train)
test_pred = xgb.predict_proba(X_test)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7644498947315362


In [14]:
scaler = MinMaxScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7643890701993251


In [17]:
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7636730277270471


In [18]:
X_train = test_pairs[cols]; y_train = test_pairs['match']
X_test = sample_pairs[cols]; y_test = sample_pairs['match']

xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train,y_train)
test_pred = xgb.predict_proba(X_test)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.8913027515916216


In [19]:
X_train = test_pairs[cols_new]; y_train = test_pairs['match']
X_test = sample_pairs[cols_new]; y_test = sample_pairs['match']

xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train,y_train)
test_pred = xgb.predict_proba(X_test)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.9008036213775513


In [20]:
sample_pairs.shape

(100000, 61)

In [21]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
Robust_col = X_train.drop(cols_new, axis=1).columns

X_train_temp = X_train.copy()
X_test_temp = X_test.copy()
#for col in tqdm(Robust_col):
#    X_test_temp[col] = X_test_temp[col] / 600000*4871594
    
xgb = XGBClassifier(nthread = -1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7718455527647068


In [22]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
import lightgbm as lgb
#clf = lgb.LGBMClassifier()

In [23]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = lgb.LGBMClassifier()
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7593659033378484


xgboost: 0.77

ada boost: 0.69

gradient boost: 0.715

random forest: 0.74

logistic: 0.62

catboost: 0.785

lgb: 0.76 (fast)

In [24]:
xgb_parameters = {'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror','binary:logistic']}

In [25]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = XGBClassifier(nthread = -1, objective = 'binary:logistic', n_estimators = 800, learning_rate = 0.1)
xgb.fit(X_train_temp,y_train)
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7772470027182486


nthread = -1, objective = 'binary:logistic'

In [26]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)

for i in [400,800,1600]:
    xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
                  importance_type='gain', objective = 'binary:logistic',
                  learning_rate=0.1, max_delta_step=0, max_depth=3,
                  min_child_weight=2, n_estimators=1600, n_jobs=-1, nthread=-1, num_parallel_tree=1,
                  random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                  subsample=0.9, tree_method='exact', validate_parameters=1,
                  verbosity=0)
    xgb.fit(X_train_temp,y_train)
    test_pred = xgb.predict_proba(X_test_temp)
    print(roc_auc_score(y_test, test_pred[:, 1]))

0.7778010399730126
0.7778010399730126
0.7778010399730126


xgb = XGBClassifier(nthread = -1, objective = 'binary:logistic', n_estimators = 800, learning_rate = 0.1)

xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
                  importance_type='gain', objective = 'binary:logistic',
                  learning_rate=0.1, max_delta_step=0, max_depth=3,
                  min_child_weight=2, n_estimators=1600, n_jobs=-1, nthread=-1, num_parallel_tree=1,
                  random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                  subsample=0.9, tree_method='exact', validate_parameters=1,
                  verbosity=0)

In [28]:
X_train = train_pairs[cols]; y_train = train_pairs['match']
X_test = test_pairs[cols]; y_test = test_pairs['match']

In [29]:
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)
xgb = CatBoostClassifier(eval_metric = 'Accuracy', depth = 5,verbose=False)
xgb.fit(X_train_temp, y_train,
       eval_set=(X_test_temp, y_test))
test_pred = xgb.predict_proba(X_test_temp)
print(roc_auc_score(y_test, test_pred[:, 1]))

0.7830721076808007


In [32]:
scaler = RobustScaler()
scaler.fit(X_train,y_train)
X_train_temp = scaler.transform(X_train)
X_test_temp = scaler.transform(X_test)

xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
                  importance_type='gain', objective = 'binary:logistic',
                  learning_rate=0.1, max_delta_step=0, max_depth=3,
                  min_child_weight=2, n_estimators=4000, n_jobs=-1, nthread=-1, num_parallel_tree=1,
                  random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                  subsample=0.9, tree_method='exact', validate_parameters=1,eval_metric = 'auc',
                  verbosity=0)

xgb.fit(X_train_temp, y_train, eval_set=[(X_train_temp, y_train),(X_test_temp, y_test)],early_stopping_rounds=100)
test_pred = xgb.predict_proba(X_test_temp)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-auc:0.92294	validation_1-auc:0.61205
[1]	validation_0-auc:0.93381	validation_1-auc:0.63093
[2]	validation_0-auc:0.94101	validation_1-auc:0.64044
[3]	validation_0-auc:0.94834	validation_1-auc:0.63666
[4]	validation_0-auc:0.94872	validation_1-auc:0.63699
[5]	validation_0-auc:0.94980	validation_1-auc:0.64219
[6]	validation_0-auc:0.95072	validation_1-auc:0.64076
[7]	validation_0-auc:0.95156	validation_1-auc:0.65783
[8]	validation_0-auc:0.95346	validation_1-auc:0.65945
[9]	validation_0-auc:0.95544	validation_1-auc:0.66305
[10]	validation_0-auc:0.95560	validation_1-auc:0.66003
[11]	validation_0-auc:0.95598	validation_1-auc:0.66211
[12]	validation_0-auc:0.95660	validation_1-auc:0.66390
[13]	validation_0-auc:0.95718	validation_1-auc:0.67217
[14]	validation_0-auc:0.95880	validation_1-auc:0.67149
[15]	validation_0-auc:0.95904	validation_1-auc:0.67281
[16]	validation_0-auc:0.95904	validation_1-auc:0.67260
[17]	validation_0-auc:0.95918	validation_1-auc:0.67463
[18]	validation_0-au

In [33]:
test_pred[:,1]

array([0.0432089 , 0.79888326, 0.0822769 , ..., 0.6307083 , 0.976035  ,
       0.46166387], dtype=float32)

In [34]:
test_pred_temp = [ 0 if x <= 0.8 else 1 for x in test_pred[:,1].tolist()]
confusion_matrix(y_test,test_pred_temp)

array([[22201,  8735],
       [21226, 47838]])